<h1>Smart Tour Guide Bot</h1>

**Project Title:**
Multimodal AI Travel Assistant

**Names:**

- Renad Naser
- ⁠Amjad Althagafi

**Objective:**

In a world with many transportations and countries, many people are becoming excited about the visiting new places and discover new things. 
Nowadays, the internet is full with many resources which make it difficult to find a specific information with being sure that it is correct or enough.

So we design and deploy an intelligent AI chatbot that answers user travel-related questions using voice or text, by extracting and summarizing data from global travel guide sources.

**Description:**

This project aims to build a multimodal RAG chatbot that allows users to ask questions and requests, like —“What can I visit in Paris in 2 days?”—and receive instant, accurate responses. The chatbot uses speech recognition, and summarize relevant information from tourist guides like Wikivoyage. It supports both text and voice input and provides responses using pre-trained transformer models.

**AI Techniques:**

	•	Speech Recognition (Whisper by OpenAI)
	•	Text Chunking & Embedding (LangChain + HuggingFace Transformers)
	•	Vector Search (ChromaDB or FAISS DB)
	•	Multimodal RAG system
	•	Conversational Agents with Memory (LangChain Agents)
	•	LangSmith for evaluation

**Applications:**

	•	Smart travel planning assistants
	•	Accessibility tools for visually/hearing-impaired travelers
	•	Educational tools for learning about geography and culture
	•	Chatbots for tourism websites or mobile travel apps

In [1]:
!pip install faiss-cpu pyowm

In [7]:
import os
from langchain_core.documents import Document
from typing import Dict, List, Any
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.tools import Tool
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.utilities import OpenWeatherMapAPIWrapper,WikipediaAPIWrapper

In [3]:
from dotenv import load_dotenv
_ = load_dotenv()


step 1 : creat wiki function.


In [ ]:
weather_api = OpenWeatherMapAPIWrapper()  #https://home.openweathermap.org/api_keys

wiki_api = WikipediaAPIWrapper()


'In Riyadh, the current weather is as follows:\nDetailed status: overcast clouds\nWind speed: 4.12 m/s, direction: 119°\nHumidity: 9%\nTemperature: \n  - Current: 32.32°C\n  - High: 32.32°C\n  - Low: 32.32°C\n  - Feels like: 30.04°C\nRain: {}\nHeat index: None\nCloud cover: 100%'

In [ ]:
def docs(article):
  return Document(
      page_content=article['text'],
      metadata={"title": article['title']}
  )

documents = []
for raw in data:
 documents.append(docs(raw))

documents
 

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents()#.split_text(data)
split_docs

In [15]:
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
)

In [21]:
def wiki_search(query):
    data = wiki_api.load(query)
    split_docs =  splitter.split_documents(data)
    
    return split_docs

In [ ]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(split_docs, embed)

# 5. Create retriever
retriever = vectorstore.as_retriever()

In [31]:
vectorstore.save_local("faiss.index")

In [32]:
from langchain.llms import OpenAI
# from google.colab import userdata
# api_key = userdata.get('OPENAI_API')
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct")  # or "gpt-4"


In [33]:
# retrieval method 1
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm ,  # Swap with HuggingFaceHub or others if needed
    retriever=retriever,
    return_source_documents=True
)

In [34]:
query = query = "What is paris?"
result = qa_chain({"query": query})

print("Answer:\n", result["result"])
print("\nSources:\n", result["source_documents"])

Answer:
  Paris is the capital and largest city of France.

Sources:
 [Document(id='48bdb864-78d1-4957-a0b6-d82d6a6c906b', metadata={'title': 'April'}, page_content='London, United Kingdom\n Madrid, Spain\n Paris, France\n Rotterdam, Netherlands\n Utrecht, Netherlands\n Zurich, Switzerland'), Document(id='1f936bc6-b39e-4328-9e71-2dd0533d321f', metadata={'title': 'Algebra'}, page_content='History'), Document(id='ce24d367-c2e5-4399-b844-ff549dba2e27', metadata={'title': 'Afghanistan'}, page_content='History'), Document(id='7bc39256-2a55-47d0-b7ef-aa29f65fa819', metadata={'title': 'Armenia'}, page_content='History')]


In [35]:
query = "can you make a trip plan for 2 days in paris?"
result = qa_chain({"query": query})

print("Answer:\n", result["result"])
print("\nSources:\n", result["source_documents"])


Answer:
  I'm sorry, I cannot make a trip plan as I am a text-based program. However, I can provide information about events and locations in Paris.

Sources:
 [Document(id='48bdb864-78d1-4957-a0b6-d82d6a6c906b', metadata={'title': 'April'}, page_content='London, United Kingdom\n Madrid, Spain\n Paris, France\n Rotterdam, Netherlands\n Utrecht, Netherlands\n Zurich, Switzerland'), Document(id='8785d90c-3ab9-4d1a-a6e5-5017a7e17977', metadata={'title': 'December'}, page_content='Moveable and Non-Single Day Events'), Document(id='cdb38401-2449-42ef-82f5-99a56417740f', metadata={'title': 'August'}, page_content='Moveable and Monthlong events'), Document(id='0788060d-86b0-4402-8cba-a36865fa22ac', metadata={'title': 'Austria'}, page_content='Food')]


In [ ]:
# import openai

# # Set your API key
# openai.api_key = "your-api-key"

# # === Step 1: Function to classify and extract topic ===
# def classify_request(user_input: str) -> tuple:
#     prompt = f"""
# Classify the user's request as one of the following categories:
# - "weather"
# - "wikipedia"
# - "other"

# Then extract the main topic or location.

# User request: "{user_input}"
# Category:
# Topic:
# """

#     response = openai.ChatCompletion.create(
#         model="gpt-3.5-turbo",
#         temperature=0,
#         messages=[{"role": "user", "content": prompt}],
#         max_tokens=60
#     )

#     content = response.choices[0].message["content"].strip()

#     # Simple parsing
#     lines = content.splitlines()
#     category = ""
#     topic = ""
#     for line in lines:
#         if line.lower().startswith("category"):
#             category = line.split(":")[1].strip().lower()
#         elif line.lower().startswith("topic"):
#             topic = line.split(":")[1].strip()
#     return category, topic


# # === Step 2: Simulated API calls ===
# def call_weather_api(location: str):
#     data = weather_api.run(location)
#     # temperature and humidity
#     return f"Temperature and humidity in {location}: {data}"

# def call_wikipedia_api(topic: str):
#     data = wiki_api.run(topic)
#     return f"Here's a summary about {topic}: '...'"  # mock response


# # === Step 3: Router ===
# def handle_user_request(user_input: str) -> str:
#     category, topic = classify_request(user_input)

#     if category == "weather":
#         return call_weather_api(topic)
#     elif category == "wikipedia":
#         return call_wikipedia_api(topic)
#     else:
#         return "Sorry, I can't help with that type of request yet."


# # === Step 4: Test Example ===
# if __name__ == "__main__":
#     user_input = input("User: ")
#     result = handle_user_request(user_input)
#     print("Agent:", result)
